In [1]:
import pandas as pd
import regex as re
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk import pos_tag,pos_tag_sents
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn import svm
from sklearn import metrics
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split  
from sklearn.model_selection import GridSearchCV
import pickle
#BernoulliNB

In [2]:
#import data
#input_data= pd.read_csv("C:\\Users\\SHELLY_BENZY\\Documents\\Training\\Data_Science\\Data_Mining_Data_Pipeline\\Naive Bayes\\deceptive-opinion.csv")
input_data= pd.read_csv("deceptive-opinion.csv")

In [3]:
#Shape & describe
input_data.shape

(1600, 5)

In [4]:
#Shape & describe
input_data.describe(include='all')

,deceptive,hotel,polarity,source,text
count,1600,1600,1600,1600,1600
unique,2,20,2,3,1596
top,deceptive,conrad,negative,MTurk,Very disappointed in our stay in Chicago Monoc...
freq,800,80,800,800,2


In [5]:
#Info of Dataframe
input_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600 entries, 0 to 1599
Data columns (total 5 columns):
deceptive    1600 non-null object
hotel        1600 non-null object
polarity     1600 non-null object
source       1600 non-null object
text         1600 non-null object
dtypes: object(5)
memory usage: 62.6+ KB


In [6]:
input_data.head(5)

,deceptive,hotel,polarity,source,text
0,truthful,conrad,positive,TripAdvisor,We stayed for a one night getaway with family ...
1,truthful,hyatt,positive,TripAdvisor,Triple A rate with upgrade to view room was le...
2,truthful,hyatt,positive,TripAdvisor,This comes a little late as I'm finally catchi...
3,truthful,omni,positive,TripAdvisor,The Omni Chicago really delivers on all fronts...
4,truthful,hyatt,positive,TripAdvisor,I asked for a high floor away from the elevato...


In [7]:
input_data['org_text']=input_data['text']

In [8]:
#Change to Lower Case
input_data['text']= input_data['text'].str.lower()

In [9]:
#Remove URL
input_data['text'] = input_data['text'].str.replace('http\S+|www.\S+', ' ', case=False)

In [10]:
#Remove Numbers
input_data['text']=input_data['text'].str.replace('\d+', ' ')

In [11]:
#Remove special characters
input_data['text']=input_data['text'].str.replace('\W', ' ')

In [12]:
#Remove double space
input_data['text']=input_data['text'].str.replace('\s{3,}', ' ')
input_data['text']=input_data['text'].str.replace('\s{2,}', ' ')

In [13]:
#Lammatization using textblob
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from textblob import TextBlob, Word

def lemmatize_with_postag(sentence):
    sent = TextBlob(sentence)
    tag_dict = {"J": 'a', 
                "N": 'n',
                "V": 'v', 
                "R": 'r'}
    words_and_tags = [(w, tag_dict.get(pos[0], 'n')) for w, pos in sent.tags]  
    #dictionary.get(keyname, value)
    lemmatized_list = [wd.lemmatize(tag) for wd, tag in words_and_tags]
    return " ".join(lemmatized_list)

[nltk_data] Downloading package punkt to C:\Users\mathan
[nltk_data]     mohan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\mathan mohan\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to C:\Users\mathan
[nltk_data]     mohan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [14]:
tag_dict = {"J": 'a', 
                "N": 'n',
                "V": 'v', 
                "R": 'r'}

In [15]:
TextBlob('He is bad trainer')

TextBlob("He is bad trainer")

In [16]:
sent = TextBlob('He is bad trainer')

In [17]:
sent.tags

[('He', 'PRP'), ('is', 'VBZ'), ('bad', 'JJ'), ('trainer', 'NN')]

In [18]:
words_and_tags = [(w, tag_dict.get(pos[0], 'n')) for w, pos in sent.tags]  

In [19]:
words_and_tags

[('He', 'n'), ('is', 'v'), ('bad', 'a'), ('trainer', 'n')]

In [20]:
sent = TextBlob('He is bad trainer')
for w, pos in sent.tags :
    print(w)
    print(pos)
    print(pos[0])
    print(tag_dict.get(pos[0], 'n'))
    words_and_tags = [(w, tag_dict.get(pos[0], 'n')) for w, pos in sent.tags] 
    print(words_and_tags) 
    
for wd, tag in words_and_tags:
    print(wd.lemmatize(tag))

He
PRP
P
n
[('He', 'n'), ('is', 'v'), ('bad', 'a'), ('trainer', 'n')]
is
VBZ
V
v
[('He', 'n'), ('is', 'v'), ('bad', 'a'), ('trainer', 'n')]
bad
JJ
J
a
[('He', 'n'), ('is', 'v'), ('bad', 'a'), ('trainer', 'n')]
trainer
NN
N
n
[('He', 'n'), ('is', 'v'), ('bad', 'a'), ('trainer', 'n')]
He
be
bad
trainer


In [21]:
sent.tags

[('He', 'PRP'), ('is', 'VBZ'), ('bad', 'JJ'), ('trainer', 'NN')]

In [22]:
import nltk
nltk.download('wordnet')
print(lemmatize_with_postag("He were a very best trainer running eating institute"))

He be a very best trainer run eat institute


[nltk_data] Downloading package wordnet to C:\Users\mathan
[nltk_data]     mohan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [23]:
#Apply for Lammetization
input_data['text']=input_data.text.apply(lemmatize_with_postag)

In [24]:
#download the NLTK stop words
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\mathan
[nltk_data]     mohan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [25]:
#Create List of stop words
stop_words_nltk=nltk.corpus.stopwords.words('english')

In [26]:
# Import stopwords with scikit-learn (Stop words Can be also done with Sklearn)
from sklearn.feature_extraction import text
stop_words_sklearn = text.ENGLISH_STOP_WORDS

In [27]:
#Words to be retained and not skipped
retain_words=['i','my']

In [28]:
#remove important words if any from the stop words list
for word in retain_words:
    #stop_words_sklearn.remove(word)
    stop_words_nltk.remove(word)

In [29]:
#Add skip words or additional stop words
additional_stopwords = ['stop','rare','xyz']
for word in additional_stopwords:
    stop_words_nltk.extend(additional_stopwords)

In [30]:
#Remove the stop words from the text
for word in stop_words_nltk :
    input_data['text'] = input_data['text'].replace(to_replace=r'\b%s\b'%word, value="",regex=True)

In [31]:
#Remove leading and trailing spaces
input_data['text']=input_data['text'].map(lambda x: x.strip())

In [32]:
#POS tagging
def pos(text):
    return TextBlob(text).tags
    #return[(word,tag) for word, tag in TextBlob(text).tags if tag.startswith('NN') or tag.startswith('VB')]

In [33]:
#TextBlob("this is a good day").tags
pos("this is a good day")

[('this', 'DT'), ('is', 'VBZ'), ('a', 'DT'), ('good', 'JJ'), ('day', 'NN')]

In [34]:
#Apply for POS
pos_output=input_data.text.apply(pos)

In [35]:
pos_output

0       [(stay, VB), (one, CD), (night, NN), (getaway,...
1       [(triple, JJ), (rate, NN), (upgrade, JJ), (vie...
2       [(come, JJ), (little, JJ), (late, JJ), (i, NN)...
3       [(omni, NNS), (chicago, VBP), (really, RB), (d...
4       [(i, NN), (ask, VBP), (high, JJ), (floor, NN),...
5       [(i, NN), (stay, VBP), (omni, RB), (one, CD), ...
6       [(stay, NN), (conrad, NNS), (night, NN), (than...
7       [(get, VB), (back, RB), (day, NN), (chicago, N...
8       [(arrive, JJ), (omni, NNS), (nd, JJ), (septemb...
9       [(visit, NN), (chicago, NN), (choose, NN), (hy...
10      [(i, NN), (stay, VBP), (fairmont, RB), (chicag...
11      [(ok, NN), (first, RB), (trip, NN), (chicago, ...
12      [(arrive, JJ), (friday, NN), (room, NN), (read...
13      [(my, PRP$), (wife, NN), (i, VBZ), (come, VBP)...
14      [(i, NN), (get, VBP), (sunday, JJ), (night, NN...
15      [(superb, JJ), (hotel, NN), (room, NN), (huge,...
16      [(stay, NN), (conrad, NNS), (night, NN), (frie...
17      [(conr

In [36]:
pos_output = pd.DataFrame(pos_output)

In [37]:
pos_output

,text
0,"[(stay, VB), (one, CD), (night, NN), (getaway,..."
1,"[(triple, JJ), (rate, NN), (upgrade, JJ), (vie..."
2,"[(come, JJ), (little, JJ), (late, JJ), (i, NN)..."
3,"[(omni, NNS), (chicago, VBP), (really, RB), (d..."
4,"[(i, NN), (ask, VBP), (high, JJ), (floor, NN),..."
5,"[(i, NN), (stay, VBP), (omni, RB), (one, CD), ..."
6,"[(stay, NN), (conrad, NNS), (night, NN), (than..."
7,"[(get, VB), (back, RB), (day, NN), (chicago, N..."
8,"[(arrive, JJ), (omni, NNS), (nd, JJ), (septemb..."
9,"[(visit, NN), (chicago, NN), (choose, NN), (hy..."


In [38]:
pos_output['text'].map(lambda x:" ".join(["/".join(x) for x in x ]) )

0       stay/VB one/CD night/NN getaway/NN family/NN t...
1       triple/JJ rate/NN upgrade/JJ view/NN room/NN l...
2       come/JJ little/JJ late/JJ i/NN finally/RB catc...
3       omni/NNS chicago/VBP really/RB deliver/VB fron...
4       i/NN ask/VBP high/JJ floor/NN away/RB elevator...
5       i/NN stay/VBP omni/RB one/CD night/NN follow/N...
6       stay/NN conrad/NNS night/NN thanksgiving/VBG c...
7       get/VB back/RB day/NN chicago/NN shop/NN girlf...
8       arrive/JJ omni/NNS nd/JJ september/JJ day/NN s...
9       visit/NN chicago/NN choose/NN hyatt/NNS due/JJ...
10      i/NN stay/VBP fairmont/RB chicago/RB one/CD ni...
11      ok/NN first/RB trip/NN chicago/NN i/NN litlle/...
12      arrive/JJ friday/NN room/NN ready/JJ u/JJ mind...
13      my/PRP$ wife/NN i/VBZ come/VBP spend/JJ weeken...
14      i/NN get/VBP sunday/JJ night/NN stay/NN pricel...
15      superb/JJ hotel/NN room/NN huge/JJ include/VBP...
16      stay/NN conrad/NNS night/NN friends/NNS reunio...
17      conrad

In [39]:
#Add tags with words
input_data['text'] = pos_output['text'].map(lambda x:" ".join(["/".join(x) for x in x ]) )

In [40]:
#Split train and test
review_train, review_test, label_train, label_test = train_test_split(input_data['text'],input_data['deceptive'], test_size=0.2,random_state=13)

In [41]:
#TF-IDF
tf_vect = TfidfVectorizer(lowercase = True, use_idf=True, smooth_idf=True,sublinear_tf=False, ngram_range=(2,2)) #min_df=0.1)
#use_idf-->IDF weigts
#Smooth_IDF-->Division of zero is avoided
#Sublinear---> Apply sublinear tf scaling, i.e. replace tf with 1 + log(tf).

In [42]:
#fit and transform train
X_train_tf=tf_vect.fit_transform(review_train)

In [43]:
X_train_tf

<1280x27740 sparse matrix of type '<class 'numpy.float64'>'
	with 169263 stored elements in Compressed Sparse Row format>

In [44]:
tf_vect.get_feature_names()

['aaa jj',
 'aaa nn',
 'aaahed jj',
 'aback rp',
 'ability nn',
 'able jj',
 'abruptness nn',
 'absence nn',
 'absent jj',
 'absolute jj',
 'absolute nn',
 'absolutely rb',
 'absolutly rb',
 'absurd jj',
 'absurd nn',
 'abundant jj',
 'abut nn',
 'abysmal jj',
 'abysmally rb',
 'ac in',
 'ac jj',
 'ac nn',
 'ac vbd',
 'ac vbp',
 'accent jj',
 'accent nn',
 'accept in',
 'accept jj',
 'accept nn',
 'accept vb',
 'accept vbd',
 'accept vbp',
 'acceptable jj',
 'accesible jj',
 'access nn',
 'accessibility nn',
 'accessible jj',
 'accessory nn',
 'accidentally rb',
 'accomadating vbg',
 'accomadations nns',
 'accomedations nns',
 'accomidations nns',
 'accommodate jj',
 'accommodate nn',
 'accommodate vb',
 'accommodate vbp',
 'accommodation in',
 'accommodation nn',
 'accomodate nn',
 'accomodate vb',
 'accomodated vbn',
 'accomodating nn',
 'accomodating vbg',
 'accomodatingly rb',
 'accomodations nns',
 'accomondations nns',
 'accompany nn',
 'accompany vbp',
 'accomplish nn',
 'accomp

In [45]:
#get headers
X_train_tf=pd.DataFrame(X_train_tf.todense(),columns=tf_vect.get_feature_names())

In [46]:
#ONLY transform test not fit
X_test_tf = tf_vect.transform(review_test)

In [47]:
X_test_tf =pd.DataFrame(X_test_tf.todense(),columns=tf_vect.get_feature_names())

In [48]:
X_test_tf

,aaa jj,aaa nn,aaahed jj,aback rp,ability nn,able jj,abruptness nn,absence nn,absent jj,absolute jj,...,yucky jj,yummo nn,yummy jj,yummy nn,yunan nn,yup nn,zagat nn,zest jjs,zone nn,zoo nn
0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
5,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
6,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
7,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
8,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
9,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB

In [50]:
# Training Naive bayes classifier
from sklearn.naive_bayes import MultinomialNB
model1 = MultinomialNB()
model1.fit(X_train_tf, label_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [51]:
pred_Naive=model1.predict(X_test_tf)

In [52]:
metrics.accuracy_score(label_test, pred_Naive)

0.85625

In [53]:
print(classification_report(pred_Naive, label_test))

              precision    recall  f1-score   support

   deceptive       0.91      0.83      0.87       181
    truthful       0.80      0.89      0.84       139

    accuracy                           0.86       320
   macro avg       0.85      0.86      0.86       320
weighted avg       0.86      0.86      0.86       320



In [54]:
#Save Model
from sklearn.externals import joblib

# save the model to disk
filename = 'tfidf_model.sav'
joblib.dump(tf_vect, filename)
filename
# load the model from disk
#tfidf = joblib.load(filename)
#result = tfidf.predict(X_Live)
#print(result)


C:\Anaconda\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


'tfidf_model.sav'

In [63]:
#pkl = open('mlmodel.pickle', 'rb')
#clf = pickle.load(pkl)   
#vec = open('vectorizer.pickle', 'rb')
#tf_vect = pickle.load(vec)

In [55]:
# Training Naive bayes classifier
model2 = GaussianNB()
model2.fit(X_train_tf, label_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [56]:
pred_Naive=model2.predict(X_test_tf)

In [57]:
metrics.accuracy_score(label_test, pred_Naive)

0.678125

In [58]:
print(classification_report(pred_Naive, label_test))

              precision    recall  f1-score   support

   deceptive       0.56      0.75      0.64       124
    truthful       0.80      0.63      0.71       196

    accuracy                           0.68       320
   macro avg       0.68      0.69      0.68       320
weighted avg       0.71      0.68      0.68       320



In [59]:
#Grid Serach CV for SVM
def svc_param_selection(X, y, nfolds):
    Cs = [0.001, 0.01, 0.1, 1, 10]
    gammas = [0.001, 0.01, 0.1, 1]
    param_grid = {'C': Cs, 'gamma' : gammas}
    grid_search = GridSearchCV(svm.SVC(kernel='linear'), param_grid, cv=nfolds)
    grid_search.fit(X, y)
    return grid_search.best_params_

In [60]:
#svc_param_selection(X_train_tf,label_train,2)

In [61]:
clf = svm.SVC(C=1,gamma=0.001,kernel='linear')
clf.fit(X_train_tf,label_train)
pred = clf.predict(X_test_tf)

In [62]:
metrics.accuracy_score(label_test, pred)

0.89375

In [63]:
confusion_matrix(label_test, pred)

array([[146,  19],
       [ 15, 140]], dtype=int64)

In [160]:
print(classification_report(label_test, pred))

              precision    recall  f1-score   support

   deceptive       0.91      0.88      0.90       165
    truthful       0.88      0.90      0.89       155

    accuracy                           0.89       320
   macro avg       0.89      0.89      0.89       320
weighted avg       0.89      0.89      0.89       320

